--- Day 5: If You Give A Seed A Fertilizer ---
You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2
52 50 48
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81.
Seed number 14 corresponds to soil number 14.
Seed number 55 corresponds to soil number 57.
Seed number 13 corresponds to soil number 13.
The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [1]:
class Map:
    def __init__(self, sourceText, destinationText, data):
        self.sourceText = sourceText
        self.destinationText = destinationText
        self.rules = self.__parseData(data)

    def __parseData(self, data):
        rules = []
        for row in data:
            parts = row.split()
            destination_start = int(parts[0])
            source_start = int(parts[1])
            length = int(parts[2])
            rules.append((source_start, source_start + length, destination_start))
        return rules

    def get_map_value(self, sourceValue):
        sourceValue = int(sourceValue)
        for source_start, source_end, destination_start in self.rules:
            if source_start <= sourceValue < source_end:
                offset = sourceValue - source_start
                return destination_start + offset
        return sourceValue

    def __str__(self):
        return ', '.join([f"{rule[0]}->{rule[2]} (len: {rule[1]-rule[0]})" for rule in self.rules])

class MapKey:
    def __init__(self, sourceText, destinationText):
        self.sourceText = sourceText
        self.destinationText = destinationText

    def __eq__(self, other):
        return self.sourceText == other.sourceText and self.destinationText == other.destinationText

    def __hash__(self):
        return hash((self.sourceText, self.destinationText))
    
class Seed:
    def __init__(self, seed_no):
        self.seed_no = seed_no
        self.seed_values = {}
        self.seed_tiny_values = {}

    def add_seed_value(self, map_key, value):
        self.seed_values[map_key] = int(value)
        self.seed_tiny_values[map_key.destinationText] = int(value)

    def get_seed_value_string(self, map_key, tiny=False):
        if tiny:
            return f"{map_key.destinationText} : {self.seed_tiny_values[map_key.destinationText]}"
        else:
            return f"{map_key.sourceText} -> {map_key.destinationText} : {self.seed_values[map_key]}"

    def get_seed_values_string(self, tiny=False):
        return ' '.join([self.get_seed_value_string(map_key, tiny) for map_key in self.seed_values])

    def __str__(self):
        return f"Seed no: {self.seed_no}, {self.get_seed_values_string(tiny=True)}"

class MapService:
    def __init__(self, linesText):
        self.linesText = linesText
        self.maps = {}
        self.seeds = []
        self.__parseLinesText()

    def __parseLinesText(self):
        sections = self.linesText.split('\n\n')
        for section in sections:
            lines = section.split('\n')
            header = lines[0].split(':')
            map_name = header[0].strip()

            if map_name == "seeds":
                self.seeds = [Seed(seed_no) for seed_no in header[1].strip().split()]
            else:
                map_name_splitted = map_name.split(" ")[0].split('-to-')
                destination_text, source_text = map_name_splitted[0], map_name_splitted[1]
                map_data = lines[1:] 
                print(destination_text, "->", source_text, map_data)
                self.maps[MapKey(destination_text, source_text)] = Map(destination_text, source_text, map_data)

    def decode_seed(self, seed, source_map_value = None, source_map_key='seed'):
        if source_map_value is None:
            source_map_value = seed.seed_no

        for map_key in self.maps:
            if map_key.sourceText == source_map_key:
                map_value = self.maps[map_key].get_map_value(source_map_value)
                seed.add_seed_value(map_key, map_value)
                self.decode_seed(seed, map_value, map_key.destinationText)
        return seed
    
    def decode_seeds(self):
        for seed in self.seeds:
            seed = self.decode_seed(seed)
    
    def print_lowest_seed_destination_value_for_key(self, key):
        print(f"Lowest {key} value is {self.__get_lowest_seed_destination_value_for_key(key)}")

    def print_seeds(self):
        for seed in self.seeds:
            print(seed)

    def __get_lowest_seed_destination_value_for_key(self, key):
        return min([seed.seed_tiny_values[key] for seed in self.seeds])
    
    def __str__(self):
        return '\n'.join([str(self.maps[mapName]) for mapName in self.maps])

# Example usage
#debug lines text
linesText = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""
map_service = MapService(linesText)
map_service.decode_seeds()
print()
map_service.print_lowest_seed_destination_value_for_key('location')
map_service.print_seeds()
#Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
#Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
#Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
#Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

seed -> soil ['50 98 2', '52 50 48']
soil -> fertilizer ['0 15 37', '37 52 2', '39 0 15']
fertilizer -> water ['49 53 8', '0 11 42', '42 0 7', '57 7 4']
water -> light ['88 18 7', '18 25 70']
light -> temperature ['45 77 23', '81 45 19', '68 64 13']
temperature -> humidity ['0 69 1', '1 0 69']
humidity -> location ['60 56 37', '56 93 4']

Lowest location value is 35
Seed no: 79, soil : 81 fertilizer : 81 water : 81 light : 74 temperature : 78 humidity : 78 location : 82
Seed no: 14, soil : 14 fertilizer : 53 water : 49 light : 42 temperature : 42 humidity : 43 location : 43
Seed no: 55, soil : 57 fertilizer : 57 water : 53 light : 46 temperature : 82 humidity : 82 location : 86
Seed no: 13, soil : 13 fertilizer : 52 water : 41 light : 34 temperature : 34 humidity : 35 location : 35


In [2]:
with open('input.txt', 'r') as file:
    linesText = file.read()

map_service = MapService(linesText)
map_service.decode_seeds()
print()
map_service.print_lowest_seed_destination_value_for_key('location')
map_service.print_seeds()

seed -> soil ['341680072 47360832 98093750', '1677587229 1836834678 160297919', '1122651749 4014790961 280176335', '2279929873 2689269992 53644948', '3916120104 1199400457 172302726', '0 381576527 58197295', '1402828084 3450816018 274759145', '3909949227 2540063154 6170877', '802918801 2384227172 155835982', '4088422830 3244271552 206544466', '958754783 1997132597 28874650', '58197295 306349987 75226540', '180784667 145454582 160895405', '2334903647 1543332738 293501940', '3699983017 2997982209 25342830', '2333574821 2687941166 1328826', '3111317969 1371703183 171629555', '2806959198 2135774873 248452299', '2766721604 717118138 40237594', '3055411497 2632034694 55906472', '2628405587 3023325039 138316017', '1837885148 757355732 442044725', '3725325847 2813358829 184623380', '3353391413 2026007247 109767626', '987629433 3962399141 10015813', '717118138 2546234031 85800663', '3282947524 2742914940 70443889', '1080275742 3972414954 42376007', '133423835 0 47360832', '3463159039 3725575163

Your puzzle answer was 227653707.

The first half of this puzzle is complete! It provides one gold star: *

--- Part Two ---
Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [ ]:
! pip install numba
! pip install numpy

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def map_seeds_gpu(seeds, rules, num_rules, destination_values):
    idx = cuda.grid(1)
    if idx < seeds.size:
        seed = seeds[idx]
        for i in range(num_rules):
            source_start, source_end, destination_start = rules[i]
            if source_start <= seed < source_end:
                seed = destination_start + (seed - source_start)
                break
        destination_values[idx] = seed

class Map:
    def __init__(self, sourceText, destinationText, data):
        self.sourceText = sourceText
        self.destinationText = destinationText
        self.rules = self.__parseData(data)

    def __parseData(self, data):
        rules = []
        for row in data:
            parts = row.split()
            destination_start = int(parts[0])
            source_start = int(parts[1])
            length = int(parts[2])
            rules.append((source_start, source_start + length, destination_start))
        return rules

class MapService:
    def __init__(self, linesText):
        self.linesText = linesText
        self.maps = {}
        self.seed_ranges = []
        self.__parseLinesText()

    def __parseLinesText(self):
        sections = self.linesText.split('\n\n')
        for section in sections:
            lines = section.split('\n')
            header = lines[0].split(':')
            map_name = header[0].strip()

            if map_name == "seeds":
                seed_pairs = header[1].strip().split()
                self.seed_ranges = [(int(seed_pairs[i]), int(seed_pairs[i+1])) for i in range(0, len(seed_pairs), 2)]
            else:
                map_name_splitted = map_name.split(" ")[0].split('-to-')
                source_text, destination_text = map_name_splitted
                map_data = lines[1:]
                self.maps[(source_text, destination_text)] = Map(source_text, destination_text, map_data)

    def process_on_gpu(self, batch_size=10000, threads_per_block = 256):
        overall_min_value = np.inf
        total_ranges = len(self.seed_ranges)

        for range_index, (start, length) in enumerate(self.seed_ranges, start=1):
            range_end = start + length
            range_min_value = np.inf

            total_batches = (length + batch_size - 1) // batch_size
            for batch_num, i in enumerate(range(start, range_end, batch_size), start=1):
                batch_end = min(i + batch_size, range_end)
                batch_seeds = np.arange(i, batch_end)
                seeds_device = cuda.to_device(batch_seeds)
                destination_values_device = cuda.device_array_like(seeds_device)

                for (source_text, destination_text), map_obj in self.maps.items():
                    rules = np.array([(source_start, source_end, destination_start) 
                                      for source_start, source_end, destination_start in map_obj.rules])
                    rules_device = cuda.to_device(rules)

                    blocks_per_grid = max(1, (seeds_device.size + (threads_per_block - 1)) // threads_per_block)
                    map_seeds_gpu[blocks_per_grid, threads_per_block](seeds_device, rules_device, rules.shape[0], destination_values_device)

                    seeds_device = destination_values_device

                batch_min_value = np.min(destination_values_device.copy_to_host())
                range_min_value = min(range_min_value, batch_min_value)
                print(f"Processing range {range_index} of {total_ranges}, range: [{start}, {range_end}) - Batch {batch_num} of {total_batches} processed. Min value in batch: {batch_min_value} - OVERAL min value [{overall_min_value}]")

            overall_min_value = min(overall_min_value, range_min_value)
            print(f"Min value for range {range_index}: {range_min_value}")

        print(f"Overall min value: {overall_min_value}")
        return overall_min_value


# Example usage
# Initialize MapService
map_service = MapService(linesText)

# Process on GPU
result = map_service.process_on_gpu()
print(result)

In [ ]:
with open('input.txt', 'r') as file:
    linesText = file.read()

# Initialize MapService
map_service = MapService(linesText)

# Process on GPU
# Note: This is Batch size of 200000000, which is 200 million seeds per batch. This setting is optimal for Google Colab Tesla T4 GPU.
# If you have a different GPU, you might need to adjust this parameter. Same applies for threads_per_block parameter.
result = map_service.process_on_gpu(batch_size=200000000, threads_per_block = 1024)
print(result)

Your puzzle answer was 78775051.

Both parts of this puzzle are complete! They provide two gold stars: **

At this point, you should return to your Advent calendar and try another puzzle.